<a href="https://colab.research.google.com/github/ankitbatra22/Gesture-Recognizer/blob/develop/gesture_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
from preprocessing import VideoFolder
import json
from torchvision.transforms import *
import cv2
from PIL import Image as im
from matplotlib import pyplot as plt
from model import Net
import torch.optim as optim
import os
import tqdm
import numpy as np


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip '/content/drive/MyDrive/20bn-jester-v1.zip'

Streaming output truncated to the last 5000 lines.
  inflating: 20bn-jester-v1/60356/00010.jpg  
  inflating: 20bn-jester-v1/60356/00011.jpg  
  inflating: 20bn-jester-v1/60356/00005.jpg  
  inflating: 20bn-jester-v1/60356/00013.jpg  
  inflating: 20bn-jester-v1/60356/00007.jpg  
  inflating: 20bn-jester-v1/60356/00006.jpg  
  inflating: 20bn-jester-v1/60356/00012.jpg  
  inflating: 20bn-jester-v1/60356/00016.jpg  
  inflating: 20bn-jester-v1/60356/00002.jpg  
  inflating: 20bn-jester-v1/60356/00003.jpg  
  inflating: 20bn-jester-v1/60356/00017.jpg  
  inflating: 20bn-jester-v1/60356/00029.jpg  
  inflating: 20bn-jester-v1/60356/00001.jpg  
  inflating: 20bn-jester-v1/60356/00015.jpg  
  inflating: 20bn-jester-v1/60356/00014.jpg  
  inflating: 20bn-jester-v1/60356/00028.jpg  
  inflating: 20bn-jester-v1/388/00025.jpg  
  inflating: 20bn-jester-v1/388/00031.jpg  
  inflating: 20bn-jester-v1/388/00019.jpg  
  inflating: 20bn-jester-v1/388/00018.jpg  
  inflating: 20bn-jester-v1/388/00030

In [6]:
with open("./configs/config.json") as dataFile:
  config = json.load(dataFile)

save_dir = os.path.join(config["output_dir"], config["model_name"])

transform = Compose([
        CenterCrop(84),
        ToTensor()
        #Normalize(mean=[0.485, 0.456, 0.406],
                  #std=[0.229, 0.224, 0.225])
    ])

In [7]:
train_data = VideoFolder(root=config['train_data_folder'],
                             csv_file_input=config['train_data_csv'],
                             csv_file_labels=config['labels_csv'],
                             clip_size=config['clip_size'],
                             nclips=1,
                             step_size=config['step_size'],
                             is_val=False,
                             transform=transform,
                             )

train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=config['batch_size'], shuffle=True,
        num_workers=config['num_workers'], pin_memory=True,
        drop_last=True)
  
val_data = VideoFolder(root=config['val_data_folder'],
                          csv_file_input=config['val_data_csv'],
                          csv_file_labels=config['labels_csv'],
                          clip_size=config['clip_size'],
                          nclips=1,
                          step_size=config['step_size'],
                          is_val=True,
                          transform=transform,
                          )

val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=config['batch_size'], shuffle=False,
    num_workers=config['num_workers'], pin_memory=True,
    drop_last=False)

In [8]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

net = Net()
optimizer = torch.optim.SGD(net.parameters(), config['lr'],
                                momentum=config['momentum'],
                                weight_decay=config['weight_decay'])

criterion = nn.CrossEntropyLoss(reduction='sum')
EPOCHS = 1

Tesla P100-PCIE-16GB
True


In [ ]:
import matplotlib.pyplot as plt


net.cuda()

val_losses = []
train_losses = []
min_valid_loss = 1e99

for epoch in range(EPOCHS):
    train_loss = 0.0
    net.train()

    print("EPOCH: ", epoch)
    for i, (input, target) in enumerate(train_loader):
      print(i)
      input, target = input.to(device = device), target.to(device = device)
      net.zero_grad()
      output = net(input)
      loss = criterion(output, target)
      train_loss += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      #print("LOSS", loss / 32)

    net.eval()
    validation_loss = 0.0

    with torch.no_grad():
      for i, (item, truth) in enumerater(valid_loader):
        item, truth = item.to(device = device), truth.to(device = device)
        out = net(item)
        valLoss = criterion(out, truth)
        validation_loss += valLoss.item()

    validation_loss /= len(valid_loader)
    train_loss /= len(train_loader)
    
    if validation_loss < min_valid_loss:
      min_valid_loss = validation_loss
      torch.save(model.save_dict(), './models/weights.pt')
      
    train_losses.append(train_loss)
    val_loss.append(validation_loss)

    print("TRAIN LOSS: ", train_loss, "VALIDATION LOSS:", validation_loss)


plt.figure(figsize=(10,5))
plt.title("Training and Validation Loss")
plt.plot(val_losses,label="val")
plt.plot(train_losses,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

#torch.save(model.state_dict(), PATH)
